# Training model for m5C motif classification 

## SVM / MLP / CNN / BiLSTM

---

Author: Jianheng Liu @ Rui Zhang's Lab, SYSU, China

Email: jhfoxliu@gmail.com

Date: Jan, 2022

## Introduction

With this notebook, we will go through the process how m5C motif classifier was trained.

The 21-nt long, m5C-centered sequences with 4 kinds of motifs are used as input.

We will firstly set up the models, then use 5 times K-fold validation to compute the metrics of each model.

When models were all fitted, we will move to next notebook to analyze which model is the best.


## Hardware

- Sytem: Ubuntu 18.04.5 LTS
- CPU: Intel(R) Xeon(R) CPU E5-2680 v3 @ 2.50GHz (12 cores)
- Disk: SAMSUNG MZ7LH960HAJR-00005 (SSD)
- RAM: 64G(32Gx2) DDR4 2666MHz
- GPU: RTX2080Ti (Driver Version: 495.29.05, CUDA Version: 11.5)


## Packages

- Python==3.7.8
- numpy==1.20.0
- scipy==1.5.1
- umap-learn==0.5.2
- sklearn==0.23.1
- matplotlib==3.2.2
- seaborn==0.10.1
- tensorflow==2.7.0
- keras==2.7.0

In [1]:
import warnings
warnings.filterwarnings('ignore')

import time
import sys, os
import numpy as np
import pandas as pd
import h5py
import pickle

import tensorflow as tf
tf.autograph.set_verbosity(0)
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding, Activation, Conv1D, Conv2D, Reshape, BatchNormalization, Dropout, Flatten, Dense, merge, Input, Lambda, InputLayer, Convolution2D, MaxPooling1D, MaxPooling2D, ZeroPadding2D, Bidirectional
from keras.layers.recurrent import LSTM
from keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from keras import regularizers

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import metrics, svm

np.random.seed(42)

## 0. Define helper functions

### 0.1 Files

In [2]:
def load_h5(fn):
    with h5py.File(fn, "r") as h5File:
        X = h5File["Sequences_1x5_onehot"][:]
        Y = h5File["Labels_category_onehot"][:]
    return X, Y

### 0.2 Metrics

In [3]:
def compute_TP_FP_TN_FN_Recall_Prec_Acc_F1(DF_confusion, tag):
    TP = DF_confusion.loc[tag, tag]
    FP = DF_confusion[tag].sum() - DF_confusion.loc[tag, tag]
    TN = DF_confusion.sum().sum() - DF_confusion[tag].sum() - DF_confusion.loc[tag].sum() + DF_confusion.loc[tag, tag] 
    FN = DF_confusion.loc[tag].sum() - DF_confusion.loc[tag, tag]
    Recall = TP/(TP+FN+0.0)
    Precision = TP/(TP+FP+0.0)
    Acc = (TP+TN)/(TP+FP+TN+FN+0.0)
    F1_score = (2*Precision*Recall)/(Precision+Recall)
    return TP, FP, TN, FN, Recall, Precision, Acc, F1_score

In [4]:
def compute_AUCROC(test_out, testY, probs=None, num=4):
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    if probs is not None:
        test_out = probs
    for i in range(num):
        fpr[i], tpr[i], _ = metrics.roc_curve(testY[:, i], test_out[:, i])
        roc_auc[i] = metrics.auc(fpr[i], tpr[i])
    return fpr, tpr, roc_auc

In [5]:
def compute_AUCROC_Prec_Recall(test_out, testY, probs=None, num=4):
    precision = dict()
    recall = dict()
    roc_auc_prec_recall = dict()
    if probs is not None:
        test_out = probs
    for i in range(num):
        precision[i], recall[i], _ = metrics.precision_recall_curve(testY[:, i], test_out[:, i])
    #    roc_auc_prec_recall[i] = metrics.auc(precision[i], recall[i])
    return precision, recall, _ # , roc_auc_prec_recall

In [6]:
def get_model_metrics(test_out, testY, targets=["NSUN6", "NSUN2", "NSUN5", "Nop2"], prefix="./", probs=None):
    num = len(targets)
    if probs is None:
        matrix = metrics.confusion_matrix(testY.argmax(axis=1), test_out.argmax(axis=1))
    else:
        # for SVM
        matrix = metrics.confusion_matrix(testY.argmax(axis=1), test_out)
        
    DF_confusion = pd.DataFrame(matrix, columns=targets, index=targets)
    DF_confusion.to_csv(prefix+"/"+"confusion_matrix.csv")
    DF_scores = pd.DataFrame(columns=["TP", "FP", "TN", "FN"], index=targets)
    for tag in targets:
        TP, FP, TN, FN, Recall, Precision, Acc, F1_score = compute_TP_FP_TN_FN_Recall_Prec_Acc_F1(DF_confusion, tag)
        DF_scores.loc[tag, "TP"] = TP
        DF_scores.loc[tag, "FP"] = FP
        DF_scores.loc[tag, "TN"] = TN
        DF_scores.loc[tag, "FN"] = FN
        DF_scores.loc[tag, "Recall"] = Recall
        DF_scores.loc[tag, "Precision"] = Precision
        DF_scores.loc[tag, "Accuracy"] = Acc
        DF_scores.loc[tag, "F1-score"] = F1_score  
    
    DF_scores.to_csv(prefix+"/scores.csv")
    fpr, tpr, roc_auc = compute_AUCROC(test_out, testY, probs=probs, num=num)
    DF_AUC = pd.DataFrame(index=targets, columns=["AUC"])
    for i, tag in zip(*[range(num), targets]):
        DF_AUC.loc[tag, "AUC"] = roc_auc[i]
        DF_TPR_FPR = pd.DataFrame(columns=["TPR", "FPR"])
        DF_TPR_FPR["FPR"] = fpr[i]
        DF_TPR_FPR["TPR"] = tpr[i]
        DF_TPR_FPR.to_csv(prefix+"/"+"{}_ROC.csv".format(tag))
    DF_AUC.to_csv(prefix+"/auc.csv")
    
    precision, recall, roc_auc_prec_recall = compute_AUCROC_Prec_Recall(test_out, testY, probs=probs, num=num)
    DF_AUC_PR = pd.DataFrame(index=targets, columns=["AUC"])
    for i, tag in zip(*[range(num), targets]):
        # DF_AUC_PR.loc[tag, "AUC"] = roc_auc[i]
        DF_TPR_FPR = pd.DataFrame(columns=["Precision", "Recall"])
        DF_TPR_FPR["Precision"] = fpr[i]
        DF_TPR_FPR["Recall"] = tpr[i]
        DF_TPR_FPR.to_csv(prefix+"/"+"{}_RECALL.csv".format(tag))
    # DF_AUC_PR.to_csv(prefix+"/"+"auc_PR.csv")
    

### 0.3 History

In [7]:
def get_keras_model_training_records(model, prefix="./"):
    model_history = model.history.history
    DF_mertics = pd.DataFrame()
    DF_mertics["MSE"] = model_history['mse']
    if "val_mse" in model_history:
        DF_mertics["VAL_MSE"] = model_history['val_mse']
    DF_mertics["MAE"] = model_history['mae']
    if "val_mae" in model_history:
        DF_mertics["VAL_MAE"] = model_history['val_mae']
    DF_mertics["MAPE"] = model_history['mape']
    if "val_mape" in model_history:
        DF_mertics["VAL_MAPE"] = model_history['val_mape']
    DF_mertics["ACC"] = model_history['acc']
    if "val_acc" in model_history:
        DF_mertics["VAL_ACC"] = model_history['val_acc']
    DF_mertics["LOSS"] = model_history['loss']
    if "val_loss" in model_history:
        DF_mertics["VAL_LOSS"] = model_history["val_loss"]
    DF_mertics.to_csv(prefix+"/"+"mertics.csv")

### 0.4 Fit model functions

In [8]:
def fit_SVM_model(model, trainX, trainY, testX=None, testY=None, save_model=True, prefix="./", model_name="m5C_motifs_SVM.pkl"):
    if os.path.isdir(prefix) == False:
        os.mkdir(prefix)
        
    time0 = time.time()
    
    print("Fiting SVM model...")
    trainX = trainX.reshape(trainX.shape[0], -1)
    model.fit(trainX, trainY.argmax(1))
    
    # save model
    if save_model == True:
        pickle.dump(model, open(prefix+"/"+model_name, 'wb'))
        print("model saved.")
    
    if testX is None or testY is None:
        testX = trainX
        testY = trainY
    else:
        testX = testX.reshape(testX.shape[0], -1)
        
    y_probs = model.decision_function(testX)
    test_out = model.predict(testX)
    
    test_out = model.predict(testX)
    get_model_metrics(test_out, testY, probs=y_probs, targets=["NSUN6", "NSUN2", "NSUN5", "Nop2"], prefix=prefix)
    
    time1 = time.time() - time0
    print("SVM finished in {} sec".format(time1))
    print("================================================")
    
    return model

In [9]:
def fit_Keras_model(model_func, trainX, trainY,
                    testX=None, testY=None,
                    save_model=True, prefix="./", model_name="m5C_motifs_model.h5",
                    batch_size=512, validation_split=0.1,
                    lr=1e-4, decay=1e-3/300,
                    patience_init=30,
                    epochs_init=500,
                    report_summary=True,
                    verbose=2):
    if os.path.isdir(prefix) == False:
        os.mkdir(prefix)
    
    time0 = time.time()
    print("Fiting {} model...".format(model_name))
    
    callbacks_list = [ModelCheckpoint(prefix+"/"+"checkpoint", monitor='val_loss', verbose = verbose,                   save_best_only=True, mode='auto'), EarlyStopping(monitor='val_loss', patience=patience_init, verbose = verbose)]
    
    opt = Adam(learning_rate=lr , decay=decay) 
    
    model = model_func(opt, report_summary=report_summary)
    
    model.fit(trainX, trainY, batch_size=batch_size, epochs=epochs_init, verbose=2, validation_split=validation_split, callbacks=callbacks_list)
    
    # get training histroy
    get_keras_model_training_records(model, prefix=prefix)  
    
    # save model
    if save_model == True:
        model.save(prefix+"/"+model_name)
        print("model saved.")
        
    # compute metrics
    if testX is None or testY is None:
        testX = trainX
        testY = trainY
    test_out = model.predict(testX)
    get_model_metrics(test_out, testY, targets=["NSUN6", "NSUN2", "NSUN5", "Nop2"], prefix=prefix)
    
    time1 = time.time() - time0
    print("{} finished in {} sec".format(model_name, time1))
    print("================================================")
    
    return model

### 0.5 K-Fold

In [10]:
def KFold_training(X, Y, prefix="./", n_foldCV=5,
                   model_func=None, fit_model_func=None,  
                   keras_model=True, model_name="CV_model.h5",
                   validation_split=0.1,batch_size=512,
                   lr=1e-4, decay=1e-3/300,
                   patience_init=30, epochs_init=500,
                   report_summary=False, verbose=False):
    
    if os.path.isdir(prefix) == False:
        os.mkdir(prefix)
    
    n_foldCV = n_foldCV #Number of Crossvalidation bins
    cv_folds = list(StratifiedKFold(n_splits=n_foldCV, shuffle=True, random_state=42).split(X,Y.argmax(1))) # Non repeating CV bins
    print("Runing K-Fold....")
    for j, (train_idx, test_idx) in enumerate(cv_folds):
        print("Step {}/{}".format(j+1, n_foldCV))
        trainX_cv = X[train_idx]
        trainY_cv = Y[train_idx]
        testX_cv = X[test_idx]
        testY_cv = Y[test_idx]
        
        if os.path.isdir("./{}/CV_{}".format(prefix, j)) == False:
            os.mkdir("./{}/CV_{}".format(prefix, j))
        
        if keras_model == True:
            model = model_func
            model = fit_model_func(model_func, trainX_cv, trainY_cv, testX=testX_cv, testY=testY_cv,
                                   prefix="./{}/CV_{}".format(prefix, j),
                                   model_name=model_name,
                                   validation_split=validation_split,
                                   batch_size=batch_size,
                                   lr=lr, decay=decay,
                                   patience_init= patience_init,
                                   epochs_init=epochs_init,
                                   report_summary=report_summary,
                                   verbose=verbose)
        else:
            model = model_func()
            model = fit_model_func(model, trainX_cv, trainY_cv, testX=testX_cv, testY=testY_cv,
                                   prefix="./{}/CV_{}".format(prefix, j),
                                   model_name=model_name)
        del model

## 1. Define models

### 1.1 SVM

In [11]:
def SVM_model(random_state=42):
    model = svm.SVC(decision_function_shape='ovr', probability=True, random_state=random_state)
    return model

### 1.2 MLP

In [12]:
def MLP_model(opt, input_shape=(21, 5), filters=[256, 128, 64], report_summary=True):
    inputs = Input(shape=input_shape)
    chanDim = -1
    dropout_init = 0.2
    x = inputs
    x = Flatten()(x)
    
    for f in filters:
        x = Dense(f, activation="relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = Dropout(dropout_init)(x)
    x = Dense(4, activation="softmax")(x)
    model = Model(inputs, x)
    if report_summary == True:
        model.summary()
    model.compile(loss="categorical_crossentropy", optimizer=opt,  metrics=['mse','mae', 'mape','acc'])
    return model

### 1.3 CNN

In [13]:
def CNN_model(opt, input_shape=(21, 5), filters=[32, 64, 128], kernel = 3, report_summary=True):
    inputs = Input(shape=input_shape)
    chanDim = -1
    dropout_init = 0.3
    x = inputs
    
    for f in filters:
        x = Conv1D(f, kernel, padding="same", kernel_regularizer=regularizers.l1_l2(l1=0.0005, l2=0.0005))(x)
        x = Activation("relu")(x)
        x = BatchNormalization(axis=chanDim)(x)
        x = MaxPooling1D(pool_size=2)(x)
    
    x = Flatten()(x)
    x = Dense(128, activation="relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(dropout_init)(x)
    x = Dense(128, activation="relu")(x)
    x = Dense(4, activation="softmax")(x)
    
    model = Model(inputs, x)
    if report_summary == True:
        model.summary()
    model.compile(loss="categorical_crossentropy", optimizer=opt,  metrics=['mse','mae', 'mape','acc'])
    return model

### 1.4 BiLSTM

In [14]:
def BiLSTM_model(opt, input_shape=(21, 5), BiLSTM_layers=256, LSTM_layers=128, report_summary=True):
    inputs = Input(shape=input_shape)
    chanDim = -1
    dropout_init = 0.3
    x = inputs
    
    x = Bidirectional(LSTM(BiLSTM_layers, return_sequences=True))(x)
    x = Flatten()(x)
    x = Dense(LSTM_layers, activation="relu")(x)
    x = BatchNormalization(axis=chanDim)(x)
    x = Dropout(dropout_init)(x) 

    x = Dense(4, activation="softmax")(x)
    model = Model(inputs, x) 
    if report_summary == True:
        model.summary()
    model.compile(loss="categorical_crossentropy", optimizer=opt,  metrics=['mse','mae', 'mape','acc'])
    return model

## 2. Test model performances by K-Fold validation

In [15]:
X, Y = load_h5("./dataset_m5C_21nt.h5")

In [16]:
trainX_len = X.shape[0]
trainX_ids = np.array(range(0, trainX_len))
np.random.shuffle(trainX_ids)
trainX = X[trainX_ids]
trainY = Y[trainX_ids]

In [17]:
print(trainX.shape)
print(trainY.shape)

(4875, 21, 5)
(4875, 4)


### 2.1 SVM

In [18]:
KFold_training(trainX, trainY, prefix="./SVM/",
               model_func=SVM_model,
               fit_model_func=fit_SVM_model,
               keras_model=False,
               n_foldCV=5,
               model_name="CV_model.h5")

Runing K-Fold....
Step 1/5
Fiting SVM model...
model saved.
SVM finished in 2.9225552082061768 sec
Step 2/5
Fiting SVM model...
model saved.
SVM finished in 2.8958749771118164 sec
Step 3/5
Fiting SVM model...
model saved.
SVM finished in 2.9412333965301514 sec
Step 4/5
Fiting SVM model...
model saved.
SVM finished in 2.912475347518921 sec
Step 5/5
Fiting SVM model...
model saved.
SVM finished in 2.8843448162078857 sec


### 2.2 MLP

In [19]:
KFold_training(trainX, trainY, prefix="./MLP/",
               model_func=MLP_model,
               fit_model_func=fit_Keras_model,
               keras_model=True,
               lr=1e-4, decay=1e-3/300,
               n_foldCV=5,validation_split=0.1,
               model_name="CV_model.h5")

Runing K-Fold....
Step 1/5
Fiting CV_model.h5 model...
Epoch 1/500
7/7 - 4s - loss: 1.7102 - mse: 0.2139 - mae: 0.3466 - mape: 173288704.0000 - acc: 0.3342 - val_loss: 1.3598 - val_mse: 0.1819 - val_mae: 0.3670 - val_mape: 183503024.0000 - val_acc: 0.4564 - 4s/epoch - 518ms/step
Epoch 2/500
7/7 - 2s - loss: 1.5098 - mse: 0.1939 - mae: 0.3257 - mape: 162848416.0000 - acc: 0.3920 - val_loss: 1.3307 - val_mse: 0.1792 - val_mae: 0.3650 - val_mape: 182475248.0000 - val_acc: 0.4564 - 2s/epoch - 231ms/step
Epoch 3/500
7/7 - 2s - loss: 1.3373 - mse: 0.1730 - mae: 0.3029 - mape: 151473472.0000 - acc: 0.4667 - val_loss: 1.2996 - val_mse: 0.1758 - val_mae: 0.3619 - val_mape: 180929104.0000 - val_acc: 0.4974 - 2s/epoch - 252ms/step
Epoch 4/500
7/7 - 2s - loss: 1.2066 - mse: 0.1578 - mae: 0.2852 - mape: 142585008.0000 - acc: 0.5142 - val_loss: 1.2700 - val_mse: 0.1722 - val_mae: 0.3582 - val_mape: 179078816.0000 - val_acc: 0.6154 - 2s/epoch - 237ms/step
Epoch 5/500
7/7 - 2s - loss: 1.0639 - mse: 0.

### 2.3 CNN

In [20]:
KFold_training(trainX, trainY, prefix="./CNN1D/",
               model_func=CNN_model,
               fit_model_func=fit_Keras_model,
               keras_model=True,
               lr=1e-4, decay=1e-3/300,
               n_foldCV=5,validation_split=0.1,
               model_name="CV_model.h5")

Runing K-Fold....
Step 1/5
Fiting CV_model.h5 model...
Epoch 1/500
7/7 - 6s - loss: 2.5744 - mse: 0.2151 - mae: 0.3697 - mape: 184857040.0000 - acc: 0.2635 - val_loss: 2.3196 - val_mse: 0.1868 - val_mae: 0.3742 - val_mape: 187124128.0000 - val_acc: 0.2333 - 6s/epoch - 883ms/step
Epoch 2/500
7/7 - 0s - loss: 2.3129 - mse: 0.1842 - mae: 0.3374 - mape: 168675456.0000 - acc: 0.3909 - val_loss: 2.3446 - val_mse: 0.1899 - val_mae: 0.3768 - val_mape: 188414304.0000 - val_acc: 0.1205 - 108ms/epoch - 15ms/step
Epoch 3/500
7/7 - 0s - loss: 2.0646 - mse: 0.1529 - mae: 0.3017 - mape: 150871728.0000 - acc: 0.5151 - val_loss: 2.3630 - val_mse: 0.1920 - val_mae: 0.3781 - val_mape: 189051936.0000 - val_acc: 0.1615 - 105ms/epoch - 15ms/step
Epoch 4/500
7/7 - 0s - loss: 1.8739 - mse: 0.1258 - mae: 0.2670 - mape: 133502320.0000 - acc: 0.6268 - val_loss: 2.3727 - val_mse: 0.1928 - val_mae: 0.3781 - val_mape: 189034240.0000 - val_acc: 0.2051 - 104ms/epoch - 15ms/step
Epoch 5/500
7/7 - 0s - loss: 1.7163 - m

### 2.4 BiLSTM

In [21]:
KFold_training(trainX, trainY, prefix="./BiLSTM/",
               model_func=BiLSTM_model,
               fit_model_func=fit_Keras_model,
               keras_model=True,
               lr=1e-3, decay=1e-3/100,
               patience_init=100, epochs_init=500,
               n_foldCV=5,validation_split=0.1,
               model_name="CV_model.h5")

Runing K-Fold....
Step 1/5
Fiting CV_model.h5 model...
Epoch 1/500
7/7 - 12s - loss: 0.4911 - mse: 0.0589 - mae: 0.1253 - mape: 62631344.0000 - acc: 0.8179 - val_loss: 0.8156 - val_mse: 0.1037 - val_mae: 0.2738 - val_mape: 136894752.0000 - val_acc: 0.9205 - 12s/epoch - 2s/step
Epoch 2/500
7/7 - 8s - loss: 0.1633 - mse: 0.0174 - mae: 0.0466 - mape: 23322622.0000 - acc: 0.9573 - val_loss: 0.6285 - val_mse: 0.0738 - val_mae: 0.2241 - val_mape: 112061360.0000 - val_acc: 0.9205 - 8s/epoch - 1s/step
Epoch 3/500
7/7 - 8s - loss: 0.1217 - mse: 0.0123 - mae: 0.0343 - mape: 17147668.0000 - acc: 0.9704 - val_loss: 0.5394 - val_mse: 0.0607 - val_mae: 0.1962 - val_mape: 98077768.0000 - val_acc: 0.9179 - 8s/epoch - 1s/step
Epoch 4/500
7/7 - 8s - loss: 0.0971 - mse: 0.0095 - mae: 0.0280 - mape: 13990433.0000 - acc: 0.9766 - val_loss: 0.5118 - val_mse: 0.0571 - val_mae: 0.1867 - val_mape: 93338616.0000 - val_acc: 0.9205 - 8s/epoch - 1s/step
Epoch 5/500
7/7 - 8s - loss: 0.0878 - mse: 0.0091 - mae: 0.02

## 3. Train models with whole input 

### 3.1 SVM

In [22]:
model = fit_SVM_model(SVM_model(), trainX, trainY, prefix="./SVM/model/", model_name="m5C_motifs_21nt_SVM.h5")
del model

Fiting SVM model...
model saved.
SVM finished in 5.760254859924316 sec


### 3.2 MLP

In [23]:
model = fit_Keras_model(MLP_model, trainX, trainY, prefix="./MLP/model/", model_name="m5C_motifs_21nt_MLP.h5",
                       validation_split=0.1, batch_size=512,
                       lr=1e-4, decay=1e-3/300, patience_init=30, epochs_init=500,
                       report_summary=True, verbose=False)
del model

Fiting m5C_motifs_21nt_MLP.h5 model...
Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 21, 5)]           0         
                                                                 
 flatten_15 (Flatten)        (None, 105)               0         
                                                                 
 dense_45 (Dense)            (None, 256)               27136     
                                                                 
 batch_normalization_40 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                                 
 dropout_25 (Dropout)        (None, 256)               0         
                                                                 
 dense_46 (Dense)            (None, 128)               32896     
                   

### 3.3 CNN

In [24]:
model = fit_Keras_model(CNN_model, trainX, trainY, prefix="./CNN1D/model/", model_name="m5C_motifs_21nt_CNN1D.h5",
                       validation_split=0.1, batch_size=512,
                       lr=1e-4, decay=1e-3/300, patience_init=30, epochs_init=500,
                       report_summary=True, verbose=False)
del model

Fiting m5C_motifs_21nt_CNN1D.h5 model...
Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_17 (InputLayer)       [(None, 21, 5)]           0         
                                                                 
 conv1d_15 (Conv1D)          (None, 21, 32)            512       
                                                                 
 activation_15 (Activation)  (None, 21, 32)            0         
                                                                 
 batch_normalization_43 (Bat  (None, 21, 32)           128       
 chNormalization)                                                
                                                                 
 max_pooling1d_15 (MaxPoolin  (None, 10, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_16 (Conv1

### BiLSTM

In [25]:
model = fit_Keras_model(BiLSTM_model, trainX, trainY, prefix="./BiLSTM/model/", model_name="m5C_motifs_21nt_BiLSTM.h5",
                       validation_split=0.1, batch_size=512,
                       lr=1e-3, decay=1e-3/100, patience_init=100, epochs_init=500,
                       report_summary=True, verbose=False)
del model

Fiting m5C_motifs_21nt_BiLSTM.h5 model...
Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 21, 5)]           0         
                                                                 
 bidirectional_5 (Bidirectio  (None, 21, 512)          536576    
 nal)                                                            
                                                                 
 flatten_17 (Flatten)        (None, 10752)             0         
                                                                 
 dense_52 (Dense)            (None, 128)               1376384   
                                                                 
 batch_normalization_47 (Bat  (None, 128)              512       
 chNormalization)                                                
                                                                 
 dropout_29 (Dro